In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

In [ ]:
# Cihaz ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

In [ ]:
# Hiperparametreler
latent_dim = 256  # Latent vektör boyutu, 100 veya 128 de sık kullanılır
lr = 0.0001       # Öğrenme oranı
batch_size = 32   # Batch büyüklüğü
epochs = 300      # Epoch sayısı
img_size = 224    # Görüntü boyutu (224x224)
ngf = 64          # Generator'deki temel filtre sayısı
ndf = 64          # Discriminator'deki temel filtre sayısı
num_channels = 1  # Görüntü kanalı sayısı (gri tonlamalı için 1)

In [ ]:
# Dönüştürmeler
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.Grayscale(num_output_channels=num_channels), # Tek kanala dönüştürme
    transforms.ToTensor(),
    transforms.Normalize([0.5] * num_channels, [0.5] * num_channels), # Tek kanal için normalizasyon
])

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            self.block(latent_dim, 1024, 4, 1, 0),    # 1x1 → 4x4
            self.block(1024, 512, 4, 2, 1),           # 4x4 → 8x8
            self.block(512, 256, 4, 2, 1),            # 8x8 → 16x16
            self.block(256, 128, 4, 2, 1),            # 16x16 → 32x32
            self.block(128, 64, 4, 2, 1),             # 32x32 → 64x64
            self.block(64, 32, 4, 2, 1),              # 64x64 → 128x128
            self.block(32, 16, 4, 2, 1),              # 128x128 → 256x256
            self.block(16, 8, 4, 2, 1),               # 256x256 → 512x512
            nn.ConvTranspose2d(8, 3, 4, 2, 1),        # 512x512 → 1024x1024
            nn.Tanh()


            # Giriş: latent_dim x 1 x 1
            self._block(latent_dim, ngf * 8, 7, 1, 0),  # latent_dim -> ngf*8, 1x1 -> 7x7
            self._block(ngf * 8, ngf * 4, 4, 2, 1),    # ngf*8 -> ngf*4,  7x7 -> 14x14
            self._block(ngf * 4, ngf * 2, 4, 2, 1),    # ngf*4 -> ngf*2,  14x14 -> 28x28
            self._block(ngf * 2, ngf,     4, 2, 1),    # ngf*2 -> ngf,    28x28 -> 56x56
            self._block(ngf,     ngf // 2,4, 2, 1),    # ngf   -> ngf//2,  56x56 -> 112x112
            # Son katman: Aktivasyon olarak Tanh, num_channels'a çıktı
            nn.ConvTranspose2d(ngf // 2, num_channels, kernel_size=4, stride=2, padding=1, bias=False),
            # ngf//2 -> num_channels, 112x112 -> 224x224
            nn.Tanh()
        )

    def block(self, in_c, out_c, k, s, p):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, k, s, p, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(True)
        )

    def forward(self, z):
        return self.main(z)

In [ ]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            self.block(3, 8, 4, 2, 1),       # 1024x1024 → 512x512
            self.block(8, 16, 4, 2, 1),      # 512 → 256
            self.block(16, 32, 4, 2, 1),
            self.block(32, 64, 4, 2, 1),
            self.block(64, 128, 4, 2, 1),
            self.block(128, 256, 4, 2, 1),
            self.block(256, 512, 4, 2, 1),
            self.block(512, 1024, 4, 2, 1),  # 8x8
            nn.Conv2d(1024, 1, 4, 1, 0),     # 8x8 → 1x1
            nn.Sigmoid()


            # Giriş: num_channels x 224 x 224
            self._block(num_channels, ndf,       4, 2, 1, use_bn=False), # num_channels -> ndf, 224x224 -> 112x112
            self._block(ndf,          ndf * 2,   4, 2, 1),               # ndf -> ndf*2, 112x112 -> 56x56
            self._block(ndf * 2,      ndf * 4,   4, 2, 1),               # ndf*2 -> ndf*4, 56x56 -> 28x28
            self._block(ndf * 4,      ndf * 8,   4, 2, 1),               # ndf*4 -> ndf*8, 28x28 -> 14x14
            self._block(ndf * 8,      ndf * 16,  4, 2, 1),               # ndf*8 -> ndf*16, 14x14 -> 7x7

            
            nn.Conv2d(ndf * 16, 1, kernel_size=7, stride=1, padding=0, bias=False), # ndf*16 -> 1, 7x7 -> 1x1
            nn.Sigmoid()
        )
    
    def _block(self, in_c, out_c, k, s, p, use_bn=True):
        layers = [nn.Conv2d(in_c, out_c, k, s, p, bias=False)]
        if use_bn:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.main(x).view(-1)
    

    # Ağırlıkların başlatılması
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)

In [ ]:
# Model ve optimizasyon
G = Generator(latent_dim, ngf, num_channels).to(device)
D = Discriminator(ndf, num_channels).to(device)

G.apply(weights_init)
D.apply(weights_init)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:
print("Eğitim başlıyor...")

# Görüntülerin bulunduğu dizin
dataset = "generated_images"

# Klasördeki alt klasör sayısını al
total_foto = len(os.listdir(dataset))
count = 0

for folder in os.listdir(dataset):

    print(f"\nİşlenen {count+1}/{total_foto}")
    print(f"İşlenen klasör: {folder}")
    count += 1

    image_folder_path = os.path.join("generated_images", folder)

    # Veri kümesi ve DataLoader
    dataset = datasets.ImageFolder(image_folder_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)


    for epoch in range(epochs):
        for i, (real_imgs, _) in enumerate(dataloader):
            real_imgs = real_imgs.to(device)
            b_size = real_imgs.size(0)

            # Gerçek ve sahte etiketler
            valid = torch.full((b_size,), 0.9, dtype=torch.float, device=device) # Yumuşak etiket
            fake = torch.full((b_size,), 0.1, dtype=torch.float, device=device)  # Yumuşak etiket

            # ---------------------
            #  Discriminator Eğitimi
            # ---------------------
            optimizer_D.zero_grad()

            # Gerçek imajlarla kayıp
            real_output = D(real_imgs)
            real_loss = criterion(real_output, valid)
            # real_loss.backward() # Gradyanları biriktir

            # Sahte imajlarla kayıp
            z = torch.randn(b_size, latent_dim, 1, 1, device=device) # (batch_size, latent_dim, 1, 1)
            fake_imgs = G(z)
            fake_output = D(fake_imgs.detach()) # Generator'ün gradyanlarını D eğitimi sırasında dondur
            fake_loss = criterion(fake_output, fake)
            # fake_loss.backward() # Gradyanları biriktir

            d_loss = real_loss + fake_loss
            d_loss.backward() # Toplam gradyanı geri yay
            optimizer_D.step()

            # -----------------
            #  Generator Eğitimi
            # -----------------
            optimizer_G.zero_grad()
            # Sahte imajları Discriminator'den geçir (bu sefer detach() YOK)
            # Generator'ün hedefi, Discriminator'ün sahte imajları gerçek olarak sınıflandırmasıdır (valid etiket)
            output = D(fake_imgs)
            g_loss = criterion(output, valid) # Generator, Discriminator'ı kandırmaya çalışır (valid etiketini hedefleyerek)
            g_loss.backward()
            optimizer_G.step()

            if i % 50 == 0: # Her 50 batch'te bir logla
                print(
                    f"[Epoch {epoch+1}/{epochs}] [Batch {i}/{len(dataloader)}] "
                    f"Loss_D: {d_loss.item():.4f} (Real: {real_loss.item():.4f}, Fake: {fake_loss.item():.4f}), "
                    f"Loss_G: {g_loss.item():.4f}"
                )

        # Her 10 epoch'ta bir ya da son epoch'ta kaydet
        if ((epoch + 1) % 10 == 0 or epoch == epochs -1) and (epoch != 0):
            with torch.no_grad():
                fixed_noise = torch.randn(batch_size, latent_dim, 1, 1, device=device) # Bir batch dolusu örnek üret
                sample_batch = G(fixed_noise).detach().cpu()
                save_image(sample_batch, f"{dataset}/{folder}/epoch_{epoch+1}.png", nrow=8, normalize=True) # nrow, bir satırda kaç resim olacağını belirler
                print(f"Epoch {epoch+1} için örnek görseller {dataset}/{folder}/epoch_{epoch+1}.png dosyasına kaydedildi.")

print("Eğitim tamamlandı.")

Kullanılan cihaz: cuda
[Epoch 0/300] [Batch 0/229] Loss_D: 1.6204, Loss_G: 16.5402
[Epoch 0/300] [Batch 10/229] Loss_D: 0.0022, Loss_G: 8.0675
[Epoch 0/300] [Batch 20/229] Loss_D: 0.0038, Loss_G: 5.6813
[Epoch 0/300] [Batch 30/229] Loss_D: 0.0235, Loss_G: 23.5663
[Epoch 0/300] [Batch 40/229] Loss_D: 0.0127, Loss_G: 17.7654
[Epoch 0/300] [Batch 50/229] Loss_D: 0.0118, Loss_G: 13.4318
[Epoch 0/300] [Batch 60/229] Loss_D: 2.2323, Loss_G: 23.1836
[Epoch 0/300] [Batch 70/229] Loss_D: 0.0013, Loss_G: 15.0371
[Epoch 0/300] [Batch 80/229] Loss_D: 8.0961, Loss_G: 21.5890
[Epoch 0/300] [Batch 90/229] Loss_D: 0.7416, Loss_G: 5.0757
[Epoch 0/300] [Batch 100/229] Loss_D: 0.0041, Loss_G: 6.8170
[Epoch 0/300] [Batch 110/229] Loss_D: 0.0931, Loss_G: 17.7357
[Epoch 0/300] [Batch 120/229] Loss_D: 0.1018, Loss_G: 8.6151
[Epoch 0/300] [Batch 130/229] Loss_D: 0.0124, Loss_G: 13.1404
[Epoch 0/300] [Batch 140/229] Loss_D: 0.1482, Loss_G: 14.7921
[Epoch 0/300] [Batch 150/229] Loss_D: 0.0010, Loss_G: 7.2520
[E